<a href="https://colab.research.google.com/github/kenpurin/pyspark/blob/main/test_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ec2f735df1889155c55e2ab5a3179a90fcf914a047e5f829f397ca7ddc59dc89
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
from pyspark.sql import SparkSession
# ในกรณีที่ไม่มี spark context อยู่ก่อนแล้ว เราก็สร้างใหม่
if (sc is None):
    sc = SparkContext(master="local[*]")

spark = SparkSession(sparkContext=sc)\
        .builder\
        .appName("How to Spark")\
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [5]:
#get data test
!git clone https://github.com/codeforamerica/ohana-api.git

Cloning into 'ohana-api'...
remote: Enumerating objects: 14334, done.
remote: Counting objects: 100% (903/903), done.
remote: Compressing objects: 100% (431/431), done.
remote: Total 14334 (delta 488), reused 804 (delta 455), pack-reused 13431
Receiving objects: 100% (14334/14334), 6.19 MiB | 14.67 MiB/s, done.
Resolving deltas: 100% (9575/9575), done.


In [6]:
#/content/ohana-api/data/sample-csv/taxonomy.csv
mtcars_rdd = sc.textFile('/content/ohana-api/data/sample-csv/taxonomy.csv')

In [9]:
df=mtcars_rdd 

In [12]:
df.dtypes

AttributeError: ignored